In [ ]:
# Data analysis based on below tutorial:
# https://www.dataquest.io/blog/tutorial-time-series-analysis-with-pandas/

# Data refers to electricity production and consumption for 2006-2017 reported, as daily totals in gigawatt-hours (GWh).

# The columns of the data file are:
# Date — The date (yyyy-mm-dd format)
# Consumption — Electricity consumption in GWh
# Wind — Wind power production in GWh
# Solar — Solar power production in GWh
# Wind+Solar — Sum of wind and solar power production in GWh

In [ ]:
# Imports
import pandas as pd

In [ ]:
# WHAT IS AN INDEX IN DATA FRAME
# Playing with Timestamp structure, as a basic foundation of Time Series

# pd.to_datetime('2018-01-15 3:45pm')
# pd.to_datetime('7/8/1952')
# pd.to_datetime('7/8/1952', dayfirst=True)
# pd.to_datetime(['2018-01-05', '7/8/1952', 'Oct 10, 1995'])
pd.to_datetime(['2/25/10', '8/6/17', '12/15/12'], format='%m/%d/%y')

In [ ]:
# Upload data
from google.colab import files
files.upload()

In [ ]:
# Read data in (look out for the name after the upload)
df = pd.read_csv('opsd_germany_daily.csv')

In [ ]:
# Check data matrix dimensions and few data samples
df.shape
df.head(3)
df.tail(3)

In [ ]:
# Check column types and SET INDEX on "Date"
df.dtypes

# "Date" column should be in "datetime64[ns]" type, but it's object instead. 
# I will try to set up an index on "Date" field anyway. 
# df = df.set_index('Date') # Needs to be an operation run once
df.dtypes
df.head(3)
df.index

# PRO TIP:
# Usually above steps for setting an index could be compressed to below line:
# df = pd.read_csv('opsd_germany_daily.csv', index_col=0, parse_dates=True)

# I will use PRO TIP method, as it gives data frame with correct index type
df = pd.read_csv('opsd_germany_daily.csv', index_col=0, parse_dates=True)
df.dtypes
df.index

In [ ]:
# Add to data frame few individual index components
df['Year'] = df.index.year
df['Month'] = df.index.month
df['Weekday Name'] = df.index.day_name()

df.sample(5, random_state=0)

In [ ]:
# READ INDEX

# Get particular point in time
df.loc['2017-08-10']

# Get range
df.loc['2014-01-20':'2014-01-22']

# Get points in time by partial date
df.loc['2012-02']
df.loc['2012']

In [ ]:
# VISUALISING TIME SERIES

# Display figures inline in Jupyter notebook
import matplotlib.pyplot as plt

# Use seaborn style defaults and set the default figure size
import seaborn as sns
sns.set(rc={'figure.figsize':(11, 4)})

In [ ]:
# PLOTTING

# Plotting based on Data Frame plot function
df['Consumption'].plot(linewidth=0.5)

# The plot() method has chosen pretty good tick locations (every two years) 
# and labels (the years) for the x-axis, which is helpful. 
# However, with so many data points, the line plot is crowded and hard to read. 

In [ ]:
# Let’s plot the data as dots instead, and also look at the Solar and Wind time series.
cols_plot = ['Consumption', 'Solar', 'Wind']
axes = df[cols_plot].plot(marker='.', alpha=0.5, linestyle=None, figsize=(11, 9), subplots=True)
for ax in axes:
  ax.set_ylabel('Daily Totals (GWh)')

In [ ]:
 # The plot above suggests there may be some weekly seasonality in Germany’s electricity consumption,
 # corresponding with weekdays and weekends. Let’s plot the time series in a single year to investigate further.

# ax = opsd_daily.loc['2017', 'Consumption'].plot()
ax = df.loc['2017', 'Consumption'].plot()
ax.set_ylabel('Daily Consumption (GWh)');
# Now, clearly we can see weekly oscilations

In [ ]:
# Another interesting feature that becomes apparent at this level of granularity is the drastic decrease
# in electricity consumption in early January and late December, during the holidays.

df.loc['2016-12':'2017-02', 'Consumption'].plot(marker='.', linestyle='-')
ax.set_ylabel('Daily Consumption (GWh)');
# As suspected, consumption is highest on weekdays and lowest on weekends.

In [ ]:
# CUSTOMIZING TIME SERIES PLOTS

# To better visualize the weekly seasonality in electricity consumption in the plot above, 
# it would be nice to have vertical gridlines on a weekly time scale (instead of on the first day of each month).
# We can customize our plot with matplotlib.dates, so let’s import that module.
import matplotlib.dates as mdates

fig, ax = plt.subplots()
ax.plot(df.loc['2016-12':'2017-02', 'Consumption'], marker='.', linestyle='-')
ax.set_ylabel('Daily Consumption (GWh)')
ax.set_title('Dec 2016 to Feb 2017 Electricity Consumption')
ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MONDAY))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))

In [ ]:
# SEASONALITY

# Boxplot chart explained:
# https://towardsdatascience.com/understanding-boxplots-5e2df7bcbd51#:~:text=A%20boxplot%20is%20a%20standardized,%2C%20and%20%E2%80%9Cmaximum%E2%80%9D).&text=It%20can%20also%20tell%20you,how%20your%20data%20is%20skewed.

# Let’s further explore the seasonality of our data with box plots, using seaborn’s boxplot() function
# to group the data by different time periods and display the distributions for each group.

# We’ll first group the data by month, to visualize yearly seasonality.
fig, axes = plt.subplots(3, 1, figsize=(11, 10), sharex=True)
for name, ax in zip(['Consumption', 'Solar', 'Wind'], axes):
  sns.boxplot(data=df, x="Month", y=name, ax=ax)
  ax.set_ylabel('GWh')
  ax.set_title(name)
  # Remove the automatic x-axis label from all but the bottom subplot
  if ax != axes[-1]:
    ax.set_xlabel('')

# Although electricity consumption is generally higher in winter and lower in summer, 
# the median and lower two quartiles are lower in December and January compared to November and February, 
# likely due to businesses being closed over the holidays. 
# We saw this in the time series for the year 2017, and the box plot confirms that this is consistent pattern throughout the years.

# While solar and wind power production both exhibit a yearly seasonality, the wind power distributions have many more outliers, 
# reflecting the effects of occasional extreme wind speeds associated with storms and other transient weather conditions.

In [ ]:
# Electricity consumption weekly seasonality exploration
sns.boxplot(data=df, x="Weekday Name", y='Consumption')

# As expected, electricity consumption is significantly higher on weekdays than on weekends. 
# The low outliers on weekdays are presumably during holidays.

In [ ]:
# FREQUENCIES

# When the data points of a time series are uniformly spaced in time (e.g., hourly, daily, monthly, etc.), 
# the time series can be associated with a frequency in pandas. For example, let’s use the date_range() function 
# to create a sequence of uniformly spaced dates from 1998-03-10 through 1998-03-15 at daily frequency.

# Sample daily frequency (see "freq" attribute with "D" value for daily)
pd.date_range('1998-03-10', '1998-03-15', freq='D')

# Hourly frequency with 8 data points
pd.date_range('2004-09-20', periods=8, freq='H')

# Now, let's take a look at time series frequency
df.index
# freq=None; This makes sense, since the index was created from a sequence of dates in our CSV file, 
# without explicitly specifying any frequency for the time series.

# If we know that our data should be at a specific frequency, we can use the DataFrame’s asfreq() method to assign a frequency. 
# If any date/times are missing in the data, new rows will be added for those date/times, which are either empty (NaN), 
# or filled according to a specified data filling method such as forward filling or interpolation.
times_samples = ['2013-02-03', '2013-02-06', '2013-02-08']
consumption_samples = df.loc[times_samples, ['Consumption']].copy()
consumption_samples

# Create frequency without filing missing bits
consumption_freq = consumption_samples.asfreq('D')
# Create a column with missings forward filled
consumption_freq['Consumption - Forward Fill'] = consumption_samples.asfreq('D', method='ffill')
consumption_freq

# In the Consumption - Forward Fill column, the missings have been forward filled,
# meaning that the last value repeats through the missing rows until the next non-missing value occurs.

In [ ]:
# RESAMPLING

# It is often useful to resample our time series data to a lower or higher frequency. 
# Resampling to a lower frequency (downsampling) usually involves an aggregation operation — for example, 
# computing monthly sales totals from daily data. The daily OPSD data we’re working with in this tutorial was downsampled 
# from the original hourly time series. Resampling to a higher frequency (upsampling) is less common and often involves 
# interpolation or other data filling method — for example, interpolating hourly weather data to 10 minute intervals 
# for input to a scientific model.

# Specify data columns we want to include
data_columns = ['Consumption', 'Wind', 'Solar', 'Wind+Solar']
df_weekly_mean = df[data_columns].resample('W').mean()
df_weekly_mean.tail(3)

print(df.shape[0])
print(df_weekly_mean.shape[0])

In [ ]:
# Let's plot daily and weekly Solar consumption on single chart

# start and end dates for extract
start, end = '2017-01', '2017-06'
# plot daily and weekly resampled time series together
fig, ax = plt.subplots()
ax.plot(df.loc[start:end, 'Solar'], marker='.', linestyle='-', linewidth=0.5, label='Daily')
ax.plot(df_weekly_mean.loc[start:end, 'Solar'], marker='o', markersize=8, linestyle='-', label='Weekly Mean Resample')
ax.set_ylabel('Solar Production GWh')
ax.legend()

In [ ]:
# Compute the monthly sums, setting the value to NaN for any month which has
# fewer than 28 days of data
df_monthly = df[data_columns].resample('M').sum(min_count=28)
df_monthly.head(3)

# Now let’s explore the monthly time series by plotting the electricity consumption 
# as a line plot, and the wind and solar power production together as a stacked area plot.
fig, ax = plt.subplots()
ax.plot(df_monthly['Consumption'], color='black', label='Consumption')
df_monthly[['Wind', 'Solar']].plot.area(ax=ax, linewidth=0)
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.legend()
ax.set_ylabel('Monthly Total (GWh)');

# At this monthly time scale, we can clearly see the yearly seasonality in each time series, 
# and it is also evident that electricity consumption has been fairly stable over time, 
# while wind power production has been growing steadily, with wind + solar power comprising an increasing 
# share of the electricity consumed.

In [ ]:
# Let’s explore this further by resampling to annual frequency and computing the ratio of Wind+Solar to Consumption for each year.

# Wind+Solar to Consumption for each year.
# Compute the annual sums, setting the value to NaN for any year which has fewer than 360 days of data
df_annual = df[data_columns].resample('A').sum(min_count=360)
# The default index of the resampled DataFrame is the last day of each year,
# ('2006-12-31', '2007-12-31', etc.) so to make life easier, set the index
# to the year component
df_annual = df_annual.set_index(df_annual.index.year)
df_annual.index.name = 'Year'
# Compute the ratio of Wind+Solar to Consumption
df_annual['Wind+Solar/Consumption'] = df_annual['Wind+Solar'] / df_annual['Consumption']
df_annual.tail(3)

# Plot from 2012 onwards, because there is no solar production data in earlier years
ax = df_annual.loc[2012:, 'Wind+Solar/Consumption'].plot.bar(color='C0')
ax.set_ylabel('Fraction')
ax.set_ylim(0, 0.3)
ax.set_title('Wind+Solar Share of Annual Electricity Consumption')
plt.xticks(rotation=0)

# We can see that wind + solar production as a share of annual electricity consumption has been increasing 
# from about 15% in 2012 to about 27% in 2017.

In [ ]:
# ROLLING WINDOWS

# Rolling window operations are another important transformation for time series data. Similar to downsampling, 
# rolling windows split the data into time windows and and the data in each window is aggregated with a function such as mean(), 
# median(), sum(), etc. However, unlike downsampling, where the time bins do not overlap and the output is at a lower frequency 
# than the input, rolling windows overlap and “roll” along at the same frequency as the data, so the transformed time series is 
# at the same frequency as the original time series.

# By default, all data points within a window are equally weighted in the aggregation, but this can be changed by specifying window 
# types such as Gaussian, triangular, and others. We’ll stick with the standard equally weighted window here.

# Let’s use the rolling() method to compute the 7-day rolling mean of our daily data. We use the center=True argument 
# to label each window at its midpoint.
df_7d = df[data_columns].rolling(7, center=True).mean()
df_7d.head(10)

# We can see that the first non-missing rolling mean value is on 2006-01-04, because this is 
# the midpoint of the first rolling window.

In [ ]:
# To visualize the differences between rolling mean and resampling, let’s update our earlier plot of January-June 2017 solar power 
# production to include the 7-day rolling mean along with the weekly mean resampled time series and the original daily data.

start, end = '2017-01', '2017-06'
# plot daily and weekly resampled time series together
fig, ax = plt.subplots()
ax.plot(df.loc[start:end, 'Solar'], marker='.', linestyle='-', linewidth=0.5, label='Daily')
ax.plot(df_weekly_mean.loc[start:end, 'Solar'], marker='o', markersize=8, linestyle='-', label='Weekly Mean Resample')
ax.plot(df_7d.loc[start:end, 'Solar'], marker='.', linestyle='-', label='7-d Rolling Mean')
ax.set_ylabel('Solar Production GWh')
ax.legend()

# We can see that data points in the rolling mean time series have the same spacing as the daily data, but the curve 
# is smoother because higher frequency variability has been averaged out. In the rolling mean time series, the peaks and troughs 
# tend to align closely with the peaks and troughs of the daily time series. In contrast, the peaks and troughs in the weekly 
# resampled time series are less closely aligned with the daily time series, since the resampled time series is at a coarser 
# granularity.

In [ ]:
# TRENDS

# Time series data often exhibit some slow, gradual variability in addition to higher frequency variability such as seasonality 
# and noise. An easy way to visualize these trends is with rolling means at different time scales.

# A rolling mean tends to smooth a time series by averaging out variations at frequencies much higher than the window size 
# and averaging out any seasonality on a time scale equal to the window size. This allows lower-frequency variations in the data 
# to be explored. Since our electricity consumption time series has weekly and yearly seasonality, let’s look at rolling means 
# on those two time scales.

# We’ve already computed 7-day rolling means, so now let’s compute the 365-day rolling mean of our data.
df_365d = df[data_columns].rolling(window=365, center=True, min_periods=360).mean()

In [ ]:
# Plot daily, 7-day rolling mean, and 365-day rolling mean time series
fig, ax = plt.subplots()
ax.plot(df['Consumption'], marker='.', markersize=2, color='0.6', linestyle='None', label='Daily')
ax.plot(df_7d['Consumption'], linewidth=2, label='7-d Rolling Mean')
ax.plot(df_365d['Consumption'], color='0.2', linewidth=3, label='Trend (365-d Rolling Mean)')

# Set x-ticks to yearly interval and add legend and labels
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.legend()
ax.set_xlabel('Year')
ax.set_ylabel('Consumption (GWh)')
ax.set_title('Trends in Electricity Consumption');

# We can see that the 7-day rolling mean has smoothed out all the weekly seasonality, while preserving the yearly seasonality. 
# The 7-day rolling mean reveals that while electricity consumption is typically higher in winter and lower in summer, there is 
# a dramatic decrease for a few weeks every winter at the end of December and beginning of January, during the holidays.

# Looking at the 365-day rolling mean time series, we can see that the long-term trend in electricity consumption is pretty flat, 
# with a couple of periods of anomalously low consumption around 2009 and 2012-2013.

In [ ]:
# Explore trends in wind and solar production

fig, ax = plt.subplots()
for name in ['Wind', 'Solar', 'Wind+Solar']:
  ax.plot(df_365d[name], label=name)
  ax.xaxis.set_major_locator(mdates.YearLocator())
  ax.set_ylim(0, 400)
  ax.legend()
  ax.set_ylabel('Production (GWh)')
  ax.set_title('Trends in Electricity Production (365-d Rolling Means)')

  # We can see a small increasing trend in solar power production and a large increasing trend in wind power production, 
  # as Germany continues to expand its capacity in those sectors.